In [2]:
from pdfminer.high_level import extract_text, extract_pages
import pdfminer.layout as pml

import json
import re


In [ ]:
text = extract_text(
    "SCHEDULE_OF_CLASSES_FALL_2023_June_1.pdf", laparams=pml.LAParams(line_margin=0, boxes_flow=None)
)
with open("data.text", "w") as file:
    file.write(text)

In [32]:
pages = extract_pages(
    "SCHEDULE_OF_CLASSES_FALL_2023_June_1.pdf", laparams=pml.LAParams(line_margin=0, boxes_flow=None)
)

allPages = []
content = []
heading = "SCHEDULE OF CLASSES - FALL 2023"

for page in pages:
    content = []
    for el in page:
        if not isinstance(el, pml.LTTextBox):
            continue

        for line in el:
            targetText = line.get_text().strip("\n")

            if targetText.lower() in heading.lower():
                break

            content.append({
                "text": targetText,
                "x0": round(line.x0),
                "y0": round(line.y0),
                "height": round(line.height)
            })

    content.pop()
    content.pop()
    allPages.append(content)

with open("raw.json", "w") as file:
    file.write(json.dumps(allPages, indent=2))

In [44]:
with open("raw.json", "r") as file:
    arr = json.loads(file.read())

d = {}


program = ""
course = ""
codeHeader = ""
code = ""
section = ""
more = ""
disc = ""
lecture = {}
lab = {}

time = ""

sectionLine = 0
courseLine = 0

Sections = []

codeHeader2nLine = False

nav = {'SECTION': 71, 'DISC': 117, 'COURSE NUMBER': 159, 'COURSE TITLE/TEACHER': 239, 'DAY/TIMES': 469}

codeFormat = "[0-9][0-9][0-9]-[0-9a-zA-Z][0-9a-zA-Z][0-9a-zA-Z]-[0-9a-zA-Z][0-9a-zA-Z]"
sectionFormat = "[0-9][0-9][0-9][0-9][0-9]"
timeFormat = "^[A-Z][A-Z]*"

def handleSectionChange(section, disc, lecture, lab, more, code, program, course, codeHeader):
    Sections.append({
        "program": program,
        "course": course,
        "code": code,
        "codeHeader": codeHeader,
        "section": section,
        "disc": disc,
        "lecture": lecture,
        "lab": lab,
        "more": more,
        
    })

for index, page in enumerate(arr):
    
    print(index)
    course = ""

    for i in page:
        if i["text"] in nav.keys():
            continue

        # Header Program
        if i["height"] == 11:

            if i["text"] != program:
                handleSectionChange(section, disc, lecture, lab, more, code, program, course, codeHeader)
                section = ""
                disc = ""
                lecture = {}
                lab = {}
                more = ""
                code = ""

            program = i["text"]

        # Course Name
        if i["height"] == 8 and i["text"].isupper() and i["x0"] == nav["SECTION"]:
            course = i["text"]


        # code header section
        if i["x0"] == 73:

            if codeHeader2nLine:
                codeHeader += f"\n{i['text']}"
            else:
                codeHeader = i["text"]

            codeHeader2nLine = True

        else:
            codeHeader2nLine = False


        # Section
        if re.match(sectionFormat, i["text"]):

            sectionLine = i["y0"]

            if i["text"] != section:
                handleSectionChange(section, disc, lecture, lab, more, code, program, course, codeHeader)
                section = ""
                disc = ""
                lecture = {}
                lab = {}
                more = ""
                code = ""

            section = i["text"]

        # DISC column
        elif i["x0"] == nav["DISC"]:
            disc = i["text"]
            courseLine = i["y0"]

        # Course number column
        elif i["x0"] == nav["COURSE NUMBER"]:
            if re.match(codeFormat, i["text"]):
                code = i["text"]

            elif re.match("Lecture", i["text"]) or re.match("Lab", i["text"]):
                pass

            else:
                more += i["text"]

        # Course Title/prof colomn
        elif i["x0"] == nav["COURSE TITLE/TEACHER"]:
            
            if courseLine == sectionLine:
                if i["y0"] == courseLine:
                    lecture["title"] = i["text"]
                else:
                    lecture["Prof"] = i["text"]

            else:
                if i["y0"] == courseLine:
                    lab["Title"] = i["text"]
                else:
                    lab["Prof"] = i["text"]

        # Day/times column
        elif i["x0"] > nav["COURSE TITLE/TEACHER"]:
            
            if courseLine == sectionLine:
                if re.match(timeFormat, i["text"]):
                    time = i["text"]
                else:
                    lecture[time] = i["text"]

            else:
                if re.match(timeFormat, i["text"]):
                    time = i["text"]
                else:
                    lab[time] = i["text"]


def cond(obj):
    if obj["section"] == "":
        return False
    return True

filtered = list(filter(cond, Sections))

data = {}

for i in filtered:
    if i["program"] not in data:
        data[i["program"]] = {}

    if i["course"] not in data[i["program"]]:
        data[i["program"]][i["course"]] = {}

    if i["code"] not in data[i["program"]][i["course"]]:
        data[i["program"]][i["course"]][i["code"]] = {}

    data[i["program"]][i["course"]][i["code"]][i["section"]] = {
        "disc": i["disc"],
        "lecture": i["lecture"],
        "lab": i["lab"],
        "more": i["more"]
    }


with open("data2.json", "w") as file:
    file.write(json.dumps(filtered, indent=2))

with open("data.json", "w") as file:
    file.write(json.dumps(data, indent=2))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127


In [55]:
a = {
    "a":"asdfasdf",
    "b":"12341234",
    "c":"kjhgfd"
}

# for i in a.items():
#     print(i)

def func(o: tuple[str, str]):
    pass

l = list(map(func, a.items()))
print(l)

[('a', 'asdfasdf'), ('b', '12341234'), ('c', 'kjhgfd')]


In [59]:
timeFormat = "[0-9][0-9][0-9][0-9]-[0-9][0-9][0-9][0-9]"

a =   "                         Lecture               Pagnucco, Katherine S.                WF             1300-1430"
b = list(filter(lambda i: i != "", a.split(" ")))

if any([re.match(timeFormat, j) for j in b]):
    print("there is date")

else:
    print("There is no date")
# section, disc, code, *title, day, time = b
# c = [section, disc, code, title, day, time]

print(b)

There is no date
['Lecture', 'Pagnucco,', 'Katherine', 'S.']


In [63]:
codeFormat = "[0-9][0-9][0-9]-[0-9a-zA-Z][0-9a-zA-Z][0-9a-zA-Z]-[0-9a-zA-Z][0-9a-zA-Z]"

with open("out.json", "r") as file:
    arr = json.loads(file.read())

    for i, asd in enumerate(arr):
        if not re.match(codeFormat, asd["codeHeader"]):
            print(i, asd)

# k = list(filter(lambda x: re.match("^[A-Z][A-Z][A-Z][A-Z]", x.keys()), arr))
# print(k)

87 {'count': 89, 'program': 'Science Courses', 'course': 'MATHEMATICS', 'code': '201-NYA-05', 'codeHeader': 'SN5) OR (70% in 201-015-50).', 'section': '00006', 'disc': 'MATH', 'lecture': {'title': 'Calculus I', 'TF\x00': '1300-1430\x00', 'prof': 'Connor, Cameron James', 'R': '1430-1630'}, 'lab': {}, 'more': ''}
88 {'count': 90, 'program': 'Science Courses', 'course': 'MATHEMATICS', 'code': '201-NYA-05', 'codeHeader': 'SN5) OR (70% in 201-015-50).', 'section': '00007', 'disc': 'MATH', 'lecture': {'title': 'Calculus I', 'TF\x00': '1000-1130\x00', 'prof': 'Connor, Cameron James', 'R': '1030-1230'}, 'lab': {}, 'more': ''}
89 {'count': 91, 'program': 'Science Courses', 'course': 'MATHEMATICS', 'code': '201-NYA-05', 'codeHeader': 'SN5) OR (70% in 201-015-50).', 'section': '00008', 'disc': 'MATH', 'lecture': {'title': 'Calculus I', 'MR\x00': '1430-1600\x00', 'prof': 'Gatien, Daniel', 'T': '1430-1630'}, 'lab': {}, 'more': ''}
90 {'count': 92, 'program': 'Science Courses', 'course': 'MATHEMATIC

In [68]:
thisArray = [
    " 201-NYA-05 for Science. Math placement level 6. Prerequisites: (70% or higher in High School Math 534 or 536 OR TS5 or ",
    " SN5) OR (70% in 201-015-50).",
]

codeHeader = ""
codeHeader2nLine = False

for i in thisArray:
    text = i.strip()
    space = len(i) - len(i.lstrip())

    print(space)

    if space == 1:
        if codeHeader2nLine:
            codeHeader += f"\n{text}"
        else:

            if text not in codeHeader:
                print("new header!")
            else:
                print("Same header")

            codeHeader = text

        codeHeader2nLine = True
    else:
        codeHeader2nLine = False

print(codeHeader)

1
new header!
1
201-NYA-05 for Science. Math placement level 6. Prerequisites: (70% or higher in High School Math 534 or 536 OR TS5 or
SN5) OR (70% in 201-015-50).
